In [1]:
from liblinear.liblinearutil import *
import numpy as np
import tqdm

data_y, data_x = svm_read_problem('training.txt')
test_y, test_x = svm_read_problem('testing.txt')

data_y = np.array(data_y)
data_x = np.array(data_x)
test_y = np.array(test_y)
test_x = np.array(test_x)

data_size = len(data_y)
training_size = 8000
validation_size = data_size - training_size

lambdas = [1000, 100, 10, 1, 0.1, 0.01]

Evals, indexes = [], []
for t in tqdm.trange(10):
    np.random.seed(t)
    
    index = np.random.choice(data_size, training_size, replace=False)
    mask = np.zeros(data_size, dtype=bool)
    mask[index] = True
    
    training_y = data_y[mask]
    training_x = data_x[mask]
    validation_y = data_y[np.logical_not(mask)]
    validation_x = data_x[np.logical_not(mask)]
    
    Eval = []
    for lambda_ in lambdas:
        C = 1/lambda_
        model = train(training_y, training_x, f"-s 6 -c {C} -B 1")
        p_labels, p_acc, p_vals = predict(validation_y, validation_x, model, "-q")
        Eval.append(100 - p_acc[0])
    index = Eval.index(min(Eval))
    Evals.append(min(Eval))
    indexes.append(index)


100%|██████████| 10/10 [00:57<00:00,  5.78s/it]


In [1]:
from liblinear.liblinearutil import *

data_y, data_x = svm_read_problem('training.txt')
test_y, test_x = svm_read_problem('testing.txt')

In [2]:
C = 1/100
g = train(data_y, data_x, f"-s 6 -c {C} -B 1")
p_labels, p_acc, p_vals = predict(test_y, test_x, g, "-q")

In [3]:
import pandas as pd

for i in range(len(p_labels)):
    p_labels[i] = p_labels[i] == 1

p_labels = pd.Series(p_labels)
data = pd.read_csv("same_season_sample_submission.csv")
data["home_team_win"] = p_labels
data.to_csv("liblinear.csv", index = False)